# Hand Gesture Detection System
---

## 1. Import necessary libraries

In [ ]:
import kagglehub
from google.colab import drive
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import random
import os
from collections import Counter
import seaborn as sns

---

## 2. Download Dataset Files

In [ ]:
# Download the dataset from KaggleHub
src_path = kagglehub.dataset_download("marusagar/hand-gesture-detection-system")
print("Path to dataset files:", src_path)

---

## 3. Mount Google Drive

In [ ]:
# Mount Google Drive to access files
drive.mount('/content/drive')

---

## 4. Set Data Directory

In [ ]:
# Define the path to your dataset in Google Drive
DATA_DIR = "/content/drive/MyDrive/Datasets/hand-gesture-detection_dataset"

---

## 5. Data Organization Setup

In [ ]:
# Define source and target root paths
source_root = Path(src_path)
target_root = Path(DATA_DIR)

# Read and rename CSV columns
train_file = pd.read_csv(source_root / "train.csv")
val_file = pd.read_csv(source_root / "val.csv")

train_file.columns = ['Image Name']
val_file.columns = ['Image Name']

# Define class ID mapping
classes = {
    0: "Left_Swipe_new",
    1: "Right_Swipe_new",
    2: "Stop_new",
    3: "Thumbs_Down_new",
    4: "Thumbs_Up_new"
}